<a href="https://colab.research.google.com/github/RamirezArizpe/Ciencia-de-datos-para-negocios/blob/Bases-de-datos-No-SQL/Parse_en_JSON_MARA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TEMA: Parse

¿Qué significa “parse”?

La palabra **parse** proviene del inglés y significa “analizar” o “descomponer”.
En programación, parsear (o hacer parsing) significa convertir una cadena de texto en una estructura de datos que el programa pueda entender y manipular.

Por ejempo: recibir datos desde una API en formato JSON como este

In [3]:
# Para comenzar a pasear y conocer qué tipo de datos tenemos en la consulta, utilizamos "type"

type({
  "nombre": "María",
  "edad": 25,
  "ciudad": "Ciudad de México"
})

# Un diccionario para python son datos estructurados clave-valor, la relación que tiene con el formato JSON es que para cada atributo ya sea en JSON o Python
# le corresponde un valor y la estructura es similar en ambos {"atributo":"valor"} y la separación de clave-valor es mediante coma (,), la relación es 1:1


dict

Cuando los datos llegan al programa, no son un diccionario ni un objeto de Python todavía, sino una cadena de texto.
Para poder trabajar con ellos, hay que parsearlos.




**Parse en Python**

Python tiene un módulo especial para esto llamado `json`.
La función más usada es json.loads(), que significa "load string", es decir, “cargar una cadena”.

In [ ]:
import json

# Cadena JSON (texto)
texto_json = '{"nombre": "María", "edad": 25, "ciudad": "Ciudad de México"}'

# Parsear: convertir texto JSON a diccionario
datos = json.loads(texto_json)




`json.loads()` convierte la cadena de texto en un diccionario de Python, que sí puede usarse para cálculos, filtros, etc.

**Parse inverso (serialización)**

El proceso inverso también existe: convertir un objeto de Python en texto JSON para enviarlo a otra aplicación o guardar en un archivo.
Eso se hace con json.dumps() (“dump string”).

In [4]:
import json

persona = {"nombre": "María", "edad": 25, "ciudad": "CDMX"}

# Convertir a texto JSON
texto = json.dumps(persona, ensure_ascii=False)
print(texto)


{"nombre": "María", "edad": 25, "ciudad": "CDMX"}


**Es importante “parsear” porque:**

los datos que se reciben desde:

APIs (como la del Metrobús),

*   APIs
*   archivos JSON
*   Servicios web

al ser texto plano, y el programa necesita parsearlos para poder:


*   extraer campos,
*   hacer calculos
*   Limpiar datos
*   Visualizar datos

**EJEMPLO PRÁCTICO USANDO UNA API**

ArcGIS es una plataforma desarrollada por Esri que permite almacenar, visualizar y analizar información geográfica.
Muchos gobiernos (como la CDMX, el INEGI o la CONABIO) publican sus datos en ArcGIS Online o en servidores ArcGIS públicos, de modo que los ciudadanos, estudiantes o investigadores puedan acceder a ellos mediante una API REST.

**¿Qué significa “API REST” en ArcGIS?**

Una API REST (Representational State Transfer) es una forma estandarizada de pedir información a un servidor mediante URLs.
En ArcGIS, cada conjunto de datos (por ejemplo, estaciones del Metrobús, rutas del Metro o cámaras de seguridad) se organiza como un Feature Service, y cada uno tiene su propia dirección (URL).

Las APIs públicas de ArcGIS permiten:

Conectar datos reales y actualizados directamente desde el gobierno.

1.   Evitar descargar manualmente archivos CSV o XLS.
2.   Trabajar con información espacial (con coordenadas geográficas)
3.   Integrar Python, R o Power BI para análisis, visualización y simulación.



Ejemplo:registros de las estaciones del Metrobús.

1. Entrar al portal donde se publica el mapa, por ejemplo:
https://serviciosatlas.sgirpc.cdmx.gob.mx/arcgis/rest/services/

2. Buscar el servicio que te interesa (en este caso “AtlasCapasPublicas/Movilidad_Integrada_CDMX”).

3. Dentro del servicio encontrarás varias capas numeradas (0, 1, 2, …).
Cada capa contiene un tipo de información:

Capa 0: Líneas del Metrobús

Capa 1: Rutas

Capa 2: Estaciones (la que usamos)

4. Al abrir la capa, verás una página con detalles técnicos y un botón o enlace llamado “Query”. Al hacer clic, ArcGIS muestra cómo consultar los datos mediante una solicitud GET, (copiaremos esa ruta url)

In [6]:
import requests # porque vamos a hacer una consulta a una API
import pandas as pd # Porque queeremos la información en formato tabla de python

# URL del servicio ArcGIS (capa de estaciones del Metrobús)
url = "https://serviciosatlas.sgirpc.cdmx.gob.mx/arcgis/rest/services/AtlasCapasPublicas/Movilidad_Integrada_CDMX/FeatureServer/2/query"

# Parámetros de la solicitud
params = {
    "where": "1=1",        # obtiene todos los registros
    "outFields": "*",       # incluye todos los campos
    "f": "json"             # formato de salida: JSON
}

# Solicitud a la API
response = requests.get(url, params=params) # a través del método GET mediante el parámetro params extraeremos todos los datos de este archivo

# Verificamos si la conexión fue exitosa
if response.status_code == 200:
    data = response.json()

    # Extraemos los datos de la sección 'features'
    estaciones = [f["attributes"] for f in data["features"]]

    # Convertimos a DataFrame
    df = pd.DataFrame(estaciones)

    # Mostramos las columnas disponibles (para explorar)
    print("Columnas disponibles:")
    print(df.columns.tolist(), "\n")

    # Mostramos algunas columnas relevantes
    columnas_interes = ["NOM_EST", "LINEA", "LATITUD", "LONGITUD"]
    columnas_presentes = [c for c in columnas_interes if c in df.columns]

    print(df[columnas_presentes].head())

else:
    print("Error en la solicitud:", response.status_code)


Columnas disponibles:
['id', 'sistema', 'nombre', 'linea', 'est', 'cve_est', 'cve_eod17', 'tipo', 'año', 'alcaldias', 'objectid'] 

Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4]


In [7]:
print(df.head(20))



    id   sistema                 nombre linea est cve_est cve_eod17  \
0    1  Metrobús                  París    07                         
1    2  Metrobús                Reforma    07                         
2    3  Metrobús               Hamburgo    07                         
3    4  Metrobús           El Ahuehuete    07                         
4    5  Metrobús               El Ángel    07                         
5    6  Metrobús               La Diana    07                         
6    7  Metrobús            Chapultepec    07                         
7    8  Metrobús       Alameda Tacubaya    07                         
8    9  Metrobús            De La Salle    07                         
9   10  Metrobús            De La Salle    07                         
10  11  Metrobús            Chapultepec    07                         
11  12  Metrobús               La Diana    07                         
12  13  Metrobús               El Ángel    07                         
13  14

In [8]:
 df.to_csv("estaciones_metrobús_cdmx.csv", index=False, encoding="utf-8-sig")